## Funzioni

## Intro

Ogni qual volta il nostro script/software necessita ripetere una certa "logica" più volte in punti diversi dello script stesso allora questo è un segnale che quella porzione di codice debba essere "impacchettata" e "generalizzata" per poter essere richiamata all'occorrenza, senza doverla riscrivere.

Questo è utile per molti motivi:
- La porzione di codice si scrive 1 sola volta.
- Più facile da manutenere. Esempio, vi accorgete che c'era un bug in quella porzione di codice oppure volete apportare una piccola modifica, a quel punto dovreste farlo in tutte le parti del software in cui viene eseguita quella stessa operazione. Nel farlo spesso succede che vi dimenticate di farlo ovunque e vi ritrovate che in alcune parti funziona e in altre no.
- Da una struttura al software e lo rende molto più leggibile. Suddividere logicamente le operazioni che vengono effettuate dal software aiuta a creare "uno scheletro" del proprio software e questo aiuta il suo sviluppo. In più impacchettare pezzi di codice e tenerli separati aiuta la leggibilità per il programmatore che deve capire cosa fa il programma.

## Struttura di una funzione
Si dichiara con "def" poi il nome che volete assegnare poi "()" poi ":"

In [9]:
def my_function():
    print("Ciao")

my_function()
pass

Ciao


Abbiamo visto come l'interprete python esegue il codice riga per riga partendo dalla prima. Quando ci sono definizioni (def) delle funzioni, lui non le esegue subito, "apprende" il fatto che ci siano e passa oltre. In questo caso la prima riga che verrà davvero eseguita sarà "my_function()" e a quel punto python "salterà" alla riga in cui è presente la funzione ed eseguirà tutte le righe al suo interno (print("ciao")) per poi tornare al punto in cui era rimasto, passando alla prossima (pass)

Le funzioni all'interno delle loro "()" possono "accettare" dei "parametri". Questo significa che quando una funzinoe viene "chiamata" sarà necessario inserire delle variabili all'interno delle "()" che la funzinoe stessa ora richiede.
Questa cosa serve per dare in input dei dati alla funzione che userà per elaborare una risposta.

In [ ]:
def my_function(testo):
    testo = testo * 3
    print(testo)

my_function("Ciao")

CiaoCiaoCiao
MaialeMaialeMaiale


I parametri possono essere più di uno. Le variabili che possiamo inserire possono essere moltepicli. Basta ricordare che: se si forniscono delle variabili come parametro alla chiamata a funzione, questi verranno presi in ordine e assegnati secondo quell'ordine:

In [13]:
def my_function(testo_1, testo_2):
    print(testo_1, testo_2)

my_function("Ciao", "come stai?")
# "Ciao" -> testo_1 e "come stai?" -> testo_2 per ordine di inserimento

Ciao come stai?


Infine le funzioni possono effettivamente restituire al chiamate delle altre variabili tramite il comando "return"

In [14]:
def my_somma(numero_1, numero_2):
    risultato = numero_1 + numero_2
    return risultato

somma = my_somma(3, 4)

print(type(somma))
print(somma)

<class 'int'>
7


Per rendere le funzioni più comprensibili all'utente, ci sono diverse cose che si possono aggiungere:
- Una documentazione che spiega cosa fa la funzione, come devono essere i parametri e cosa restituisce.
- Specificare il tipo che i parametri devono avere (numero_1 deve essere un intero)
- Specificare il tipo dell'oggetto che viene restituito (se viene restituito qualcosa) (opzionale python lo capisce da solo)

In [ ]:
def my_somma(numero_1:int, numero_2:int) -> int:
    """
    Esegue la somma tra due numeri. Sia numero_1 che numero_2 devono essere degli interi.
    La funzione restituisce la loro somma come intero.
    """
    risultato = numero_1 + numero_2
    return risultato

print(my_somma(3, 4))

# Pandas

In [ ]:
import pandas as pd

PATH_RAW = "remote_postings.csv"
PATH_CLEAN = "remote_postings_clean.csv"

df = pd.read_csv(PATH_RAW, sep=";")


# Year;IdCountry;IdCity;City;Remote_Postings_Share

df = df.groupby("IdCity")["City"].agg(set).reset_index()
df["n_city"] = df["City"].apply(lambda x: len(x))
df = df[df["n_city"] > 1]
df

# Scraping

Lo faremo con 3 Metodi:
- Scraping tramite requests della main page + parsing html (tramite BeautifulSoup).
- Scraping tramite selenium.
- Chiamate API (post)

## Requests + BeautifulSoup

Proviamo a fare scraping di qualche video su Immobiliare.it

In [1]:
import requests

from bs4 import BeautifulSoup   # Da installare con pip

Per prima cosa bisogna analizzare il sito manualmente con "Chrome DevTools". Obiettivo:
- Cercare le macro-schede che possano ricondurre a tutti i dati all'interno di _____ (se possibile)
- Identificare le query css selector sia per l'intera scheda sia per i dati al suo interno.
Man mano che abbiamo pianificato quali query useremo, iniziamo a scrivercele in delle variabili qui sotto e poi testiamo!

In [2]:
query_css = 'pre[class="lang-py s-code-block"]'

1. Usiamo requests.get(url) per richiedere al server di fornirci l'html della pagina web.
2. Ora abbiamo la sua risposta ma l'html fornito è una semplice enorme stringa, difficile da utilizzare per ricercare con le nostre query selector. Per questo motivo usiamo BeautifulSoup che è un "parser" in grado di riconoscere del codice html da una stringa e lo struttura per noi.

In [ ]:
url = "https://stackoverflow.com/questions/2612548/extracting-an-attribute-value-with-beautifulsoup"

headers= {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}
risposta = requests.get(url, headers=headers)
main_page = BeautifulSoup(risposta.text, features="html.parser")
print(main_page.prettify())

Ora che abbiamo la pagina web "parsificata" possiamo utilizzare dei metodi che BeautifulSoup ci mette a disposizione per ricercare all'interno dell'html.  
Uno dei più importanti per il nostro scopo è: .select() e .select_one().  
Select ci permette di inserire una query css selector da ricercare. Select_one cerca solo la prima occorrenza e ci restituisce solo quella, mentre select ce le mette tutte quelle che trova in una lista.  
N.B: Questa cosa è importante perché:
- **select_one** -> Restituisce un oggetto: "Tag"
- **select** -> Restituisce un oggetto: "ResultSet[Tag]", che è una specie di lista di Tag

In [ ]:
result = main_page.select('pre[class="lang-py s-code-block"]')

## Selenium

In [80]:
import os   # Usato solo per ricavare il percorso assoluto della cartella attuale del progetto
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

Iniziamo ad impostare alcune opzioni per il webdriver di chrome:  
- **headless** -> Modalità in cui non viene visualizzata la finestra del browser. Consiglio: durante la fase di test e di costruzione dello scraper, meglio non attivare questa modalità in modo da monitorare con esattezza quello che sta accadendo. Una volta ben testato si può aggiungere e far andare lo script in "background".  
- **disable-gpu** -> Tipicamente usa su windows + modalità "headless. Evita eventuali problemi dati dal tentativo di chrome di caricare la grafica anche se non dovrebbe farlo dato che abbiamo specificato "headless".  
- **window-size=1920,1080** -> Si imposta una dimensione della finestra. Impostiamo anche se siamo in modalità "headless" perché: anche se noi non la vediamo, è come se la finestra venga comunque creata e dare una dimensione fissa in cui sappiamo in anticipo quali oggetti vengono visualizzati con quella precisa dimensione ci permette di replicare con esattezza alcune situazioni e azioni (esempio, sappiamo che per trovare un pulsante non dobbiamo fare altro che attendere che la pagina venga caricata ma se la finestra viene creata più piccola, il pulsante non è visibile e necessiterebbe di uno "scroll" verso il basso!)

In [81]:
options = Options()
options.add_argument("--headless")              
options.add_argument("--disable-gpu")           
options.add_argument("--window-size=1920,1080") 

Qui abbiamo delle opzioni "extra". Non vi capiterà spesso di usarle ma in questo caso è utile:
- **add_experimental_option** -> Sono delle opzioni particolari, in questo caso aggiungiamo:
    - **download.default_directory** -> Ci permette di scegliere una cartella specifica sul pc dove i file scaricati dal nostro webdriver (il nostro falso browser) debbano essere scaricati in automatico (anziché la classica cartella "download")

In [82]:
prefs = {"download.default_directory": os.path.abspath(".")}
options.add_experimental_option("prefs", prefs)

Creiamo il nostro driver con le nostre opzioni.  
Si può creare il webdriver senza specificare nessuna opzione se non ci servono.

In [83]:
driver = webdriver.Chrome(options=options)

Ora, in modo simile a requests, effettuiamo il .get() della pagina web.  
Quello che succede è che il driver effettua la "richiesta" dell'html della pagina che gli viene restituita e lui, proprio come fa normalmente un browser, la visualizza e ovviamente la tiene in memoria dentro di sé.

In [ ]:
driver.get("https://chatgpt.it/")

Ora procediamo a trovare i vari elementi che ci servono e interagiamoci proprio come faremmo normalemente con un browser.  


Usiamo il primo come esempio per spiegare come si procede:
- **try / except** -> Sono dei comandi python che servono a impedire il crash di python se questo avviene all'interno del try.
In pratica si sta "avvertendo" python che: PROVA ad eseguire questo codice... so già che potrebbe "alzare un eccezione" (raise exeption) che solitamente blocca l'esecuzione del programma. Ma visto che ti ho avvertito che questo può succedere, anziché bloccare il programma, esegui il codice in except e continua normalmente. Questo viene fatto perché il metodo di webdriver che utilizzeremo, se non trova l'elemento richiesto è programmato per dare un eccezione ma visto che non sappiamo se un elemento verrà visualizzato con certezza o meno (esempio accettare i coockies di un sito che a volte viene visualizzato e a volte no), gli impediamo di crashare.
- **WebDriverWait** -> Dobbiamo dargli il nostro driver come parametro e un intero che corrisponde ai secondi che deve aspettare al massimo prima di arrendersi. Usiamo questa classe per dare tempo fisico al browser di visualizzare l'elemento che vogliamo cercare. Si potrebbe risolvere anche usando la libreria di python **import time** e usando il comando "time.sleep(n_secondi)" ma in quel caso aspetterebbe un numero di secondi preciso sempre mentre tramite questa sua classe, selenium aspetta massimo il numero di secondi indicato ma se lo trova prima va avanti, quindi è più efficiente.
    - **.until** -> Questo metodo di WebDriverWait ci permette di aspettare "finché" non avviene un qualcosa che specificheremo tramite parametro (esiste anche il .untilnot()).  
- **EC** -> Quest'altra classe ci serve invece per dare una "expected_conditions" quindi possiamo specificare una serie di condizioni da aspettare che si verifichino. Tipicamente usiamo:
    - **.visibility_of_element_located()** -> Che è proprio la funzione che specifica che la condizione da aspettare è che l'elemento, che richiederemo nei suoi parametri, sia visibile nel browser.
- **By** -> E' invece la classe che ci permette di specificare il tipo di "query" da utilizzare per cercare l'elemento. In questo caso specifichiamo:
    - **.CSS_SELECTOR** -> E quindi la query selector che abbiamo già trovato tramite "F12" su google chrome.  

Una volta trovato l'elemento, in base al tipo di elemento trovato si possono eseguire delle azioni.  
In questo caso, abbiamo trovato un "button" e quindi possiamo eseguire il comando **.click()** che simula il click del mouse sul tasto.

In [ ]:
try:
    button_coockies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.fc-button.fc-cta-consent'))).click()
except:
    print("Non ho trovato schermata coockies")

Capito il primo ora è facile intuire anche tutti gli altri.  
In sostanza quello che abbiamo scritto di fare è:
- Trova e inserisci testo nel box in cui si può digitare il messaggio (.send_keys(message)).  
- Trova e clicca il tasto per dare l'ok (.click()).  
- Trova e clicca il tasto per scaricare la conversazione sotto forma di .txt (.click())
Il tutto dentro il nostro try per evitare il crash del programma.  
Sarebbe più corretto inserire ogni comando all'interno di un proprio try in modo da avere un print specifico per ogni problema che può venir fuori dallo scraper. In altri casi si potrebbe decidere di non usare il try perché quella operazione è imperativa per far proseguire il programma quindi decidiamo di farlo crashare se non riesce ad eseguirla.

In [66]:
try:
    text_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
        (By.CSS_SELECTOR, '.auto-expand.wpaicg-chat-shortcode-typing'))).send_keys("Che tempo farà domani?")

    button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
        (By.CSS_SELECTOR, 'span[class="wpaicg-chat-shortcode-send"]'))).click()
    time.sleep(1)
    download_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
        (By.CSS_SELECTOR, 'span[class="wpaicg-chatbox-download-btn"]'))).click()
except:
    print("Qualcosa è andato storto!")


In [79]:
driver.quit()

In realtà si possono effettuare queste operazioni anche senza dover usare "WebDriverWait" in modo più semplice:
- .find_element -> Cerca un elemento tramite:
    - By.CSS_SELECTOR
    - O altri metodi di query (Esempio XPATH)
In questo caso funziona comunque ma ricordarsi che se la pagina è lenta a essere caricata e gli elementi non sono immediatamente visibili, find_element potrebbe non trovare l'elemento e crashare.

In [ ]:
try:
    text_box = driver.find_element(By.CSS_SELECTOR, '.auto-expand.wpaicg-chat-shortcode-typing').send_keys("Che tempo farà domani?")
    button = driver.find_element(By.CSS_SELECTOR, 'span[class="wpaicg-chat-shortcode-send"]').click()
    download_button = driver.find_element(By.CSS_SELECTOR, 'span[class="wpaicg-chatbox-download-btn"]').click()
except:
    print("Qualcosa è andato storto!")

E' possibile cercare più elementi contemporaneamente tramite:
- .find_elements -> In questo caso fare attenzione al fatto che la funzione restituisce una lista di elementi (anche se ne trova solo 1!)
oppure nel caso di EC.visibility_of_all_elements_located (elements!)

A partire da un elemento che viene trovato è possibile continuare ad utilizzarlo per cercarne degli altri al suo interno:

In [ ]:
# Solo a scopo illustrativo, non abbiamo fatto il .get() della pagina tweetter
try:
    tweets = WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'article')))    # elements!
except:
    print("No tweets trovati!")

for tweet in tweets:
    try:
        time_stamp = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
        url_tweet = tweet.find_element(By.CSS_SELECTOR,
                'a[style="color: rgb(113, 118, 123);"]').get_attribute("href")
    except:
        print("Info tweet non trovati")


## API Rest

In [85]:
message = "Ciao ti scrivo da python! Ti saluta requests e BeautifulSoup!"

In [88]:
import requests


url = "https://chatgpt.it/wp-admin/admin-ajax.php"

payload = {
    "_wpnonce": "5b7aac3794",
    "post_id": "106",
    "url": "https://chatgpt.it",
    "action": "wpaicg_chat_shortcode_message",
    "message": message,
    "bot_id": "0",
    "chatbot_identity": "shortcode",
    "wpaicg_chat_history": '[]',
    "wpaicg_chat_client_id":"8FQLM3hp6Q"
}

headers = {
    "Origin": "https://chatgpt.it",
    "Referer": "https://chatgpt.it/",
    "User-Agent": "Mozilla/5.0",
}
response = requests.post(url, data=payload, headers=headers)

re_json = response.json()
re_json["data"]


"Ciao Python! Che bello sentirti! Requests e BeautifulSoup sono due strumenti fantastici per il web scraping e l'estrazione di dati. Se hai bisogno di aiuto con qualche codice o vuoi sapere come usarli al meglio, sono qui per te! 😊"